# How to plot R2lab Radio-Maps



## Preparation





In [ ]:
# automatic reload of modules in case of changes
%load_ext autoreload
%autoreload 2

## Importing plotly

In [ ]:
import plotly
plotly.__version__

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# using plotly in offline mode is a requirement 
# in interactive mode - too slow otherwise
import plotly.offline as pyoff

pyoff.init_notebook_mode()

In [ ]:
# interactive notebook widgets 
from ipywidgets import interact, fixed, IntSlider, Dropdown

***

## Importing a few utilities to retrieve RSSI data from files and convert it to arrays

In [ ]:
# to extract data from a RSSI file
from rssi import read_rssi

In [ ]:
# convert data into x, y and z
from r2labmap import dict_to_xyzt, dict_to_3Dxyzt

## R2lab RSSI data 

Exploring a collection of `RSSI.txt` files; assume we have collected the following files on the data directory:

```
$ wc -l data/tra*/R*
   1369 trace-T1400-r1-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r1-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r1-a7-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r54-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r54-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r54-a7-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r1-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r1-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r1-a7-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r54-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r54-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r54-a7-t1-i0.008-S64-N10/RSSI.txt
```

Where

* `T500` means an emission power of 5dBm
* `T1400` means an emission power of 14dBm

* r1 means PHY rate=1Mbps
* r54 means PHY rate=54Mbps

* `a1` means 1 single valid antenna - and so 2 values in RSSI.txt
* `a3` means 2 valid antennas - and so 3 values in RSSI.txt
* `a7` means 4 valid antennas - and so 4 value in RSSI.txt

* `t1`, `i0`, `S64` and `N10` are ping parameters that are constant across the whole experiment


# Generating  2D RSSI Radio-Maps 

In [ ]:
def radiomap2D(sender, power, rate, antennas, rssi_rank):
    t_part = "T{}".format(power*100)
    r_part = "r{}".format(rate)
    # assuming antennas to be 1 3 or 7
    a_part = "a{}".format(antennas)
    filename = "data/trace-{t_part}-{r_part}-{a_part}-t1-i0.008-S64-N10/RSSI.txt"\
               .format(**locals())
    
    data_dict = read_rssi(filename, sender, rssi_rank)
    if not data_dict:
        return

    X, Y, Z, T = dict_to_xyzt(data_dict)
    rssi = go.Heatmap( x=X, y=Y, z=Z, text=T, zmin=-100, zmax=0, zauto=False, opacity=1)
    data = [rssi]
    layout = go.Layout(
        title="R2lab Radio-Map: Rx power (in dBm) when fit{:02d} is transmitting<br>from {}"
              .format(sender, filename))
    figure = go.Figure(data = data, layout=layout)
    pyoff.iplot(figure)

In [ ]:
interact(radiomap2D, 
         sender=IntSlider(min=1, max=37, step=1, value=21, continuous_update=False),
         power=Dropdown( options=[5, 14], value=5, description="Tx power in dBm"),
         rate=Dropdown(options=[1, 54], value=1, description="PHY channel rate"),
         # yeah, this is weird all right
         antennas=Dropdown(options={"1" : 1, "2": 3, "3": 7},
                           value=1,
                           description="Number of antennas: "),
         rssi_rank=IntSlider(min=0, max=3, value=0, 
                             description="RSSI rank: "),
        )

# Generating 3D RSSI Radio-Maps

In [ ]:
def radiomap3D(sender, power, rate, antennas, rssi_rank):
    
    t_part = "T{}".format(power*100)
    r_part = "r{}".format(rate)
    # assuming antennas to be 1 3 or 7
    a_part = "a{}".format(antennas)
    filename = "data/trace-{t_part}-{r_part}-{a_part}-t1-i0.008-S64-N10/RSSI.txt"\
               .format(**locals())
    
    data_dict = read_rssi(filename, sender, rssi_rank)
    if not data_dict:
        return

    X, Y, Z, T = dict_to_3Dxyzt(data_dict)
    trace = go.Surface(x=X, y=Y, z=Z, text=T)
    #,connectgaps=True
    data_test = go.Data([trace])
    axis = dict(
        showbackground=True, # show axis background                                                   
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey                          
        gridcolor="rgb(255, 255, 255)",       # set grid line color                                   
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color                              
    )
    title="R2lab Radio-Map: Rx power (in dBm) when fit{:02d} is transmitting<br>from {}".format(sender, filename)
    layout = go.Layout(
        autosize=True,
        title = title,
        scene=go.Scene(                                     
            xaxis=go.XAxis(axis), # set x-axis style                                                  
            yaxis=go.YAxis(axis), # set y-axis style                                                  
            zaxis=go.ZAxis(axis, title="RSSI (dBm)  ")  # set z-axis style                                                  
        )
    )
    
    figure = go.Figure(data=data_test, layout=layout)
    pyoff.iplot(figure)

In [ ]:
interact(radiomap3D, 
         sender=IntSlider(min=1, max=37, step=1, value=21, continuous_update=False),
         power=Dropdown( options=[5, 14], value=5, description="Tx power in dBm"),
         rate=Dropdown(options=[1, 54], value=1, description="PHY channel rate"),
         # yeah, this is weird all right
         antennas=Dropdown(options={"1" : 1, "2": 3, "3": 7},
                           value=1,
                           description="Number of antennas: "),
         rssi_rank=IntSlider(min=0, max=3, value=0, 
                             description="RSSI rank: "),
        )